In [0]:
# import xml.etree.ElementTree as ET

# my_file = 'invoice2.xml'

# tree = ET.parse(my_file)
# root = tree.getroot()

# def print_subtree(subtree):
#     for y in subtree:
#         print( "\t", y.tag, ":", y.text)

# for x in root:
#     print (x.tag, x.attrib)
#     print_subtree(x.getchildren())

In [0]:
import spacy
import pprint
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [0]:
doc = nlp('European authorities fined Google a record $5.1 billion\
on Wednesday for abusing its power in the mobile phone market and \
ordered the company to alter its practices')

print([(X.text, X.label_) for X in doc.ents])

In [0]:
print([(X, X.ent_type_) for X in doc])


In [0]:
from bs4 import BeautifulSoup
import requests
import re
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))
ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/\
peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&\
module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)
# print(article.word,article.ents)
print([(X.text, X.label_) for X in article.ents])

In [0]:
labels = [x.label_ for x in article.ents]
set(labels)

In [0]:
items = [x.text for x in article.ents]
Counter(items).most_common(3)

In [0]:
sentences = [x for x in article.sents]
print(sentences[20])

In [0]:
displacy.render(nlp(str(sentences[20])), jupyter=True, style='ent')


In [0]:
# displacy.render(nlp(str(sentences[20])), style='dep', jupyter = True, options = {'distance': 50})

In [0]:
dict([(str(x), x.label_) for x in nlp(str(sentences[20])).ents])

In [0]:
print([(x, x.ent_iob_, x.ent_type_) for x in sentences[20]])

In [0]:
# !pip install stanfordnlp

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
df.isnull().sum()


In [0]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

In [0]:
df.groupby('Tag').size().reset_index(name='counts')


In [0]:
X = df.drop('Tag', axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)
X_train.shape, y_train.shape

In [0]:
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)

In [0]:
new_classes = classes.copy()
new_classes.pop()
new_classes

In [0]:
print(classification_report(y_pred=per.predict(X_test), y_true=y_test, labels=new_classes))

In [0]:
sgd = SGDClassifier()
sgd.partial_fit(X_train, y_train, classes)